# Predicting Satisfiability of SAT-3 Problems
## Solving the Classification Task using a ***Long Short-Term Memory (LSTM)*** 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
__counter__ = random.randint(0,2e9)

from IPython.display import HTML, display

In [3]:
import os, shutil
import json

from tuning import tune_parameters
from data_loader import dataset_processing
from train import training, testing

Device is: cuda:0


### Training and Evaluation process

In order to train the model, a *training set* is used alongside with a *validation set*. The latter is used to measure the performance of the model. The training and validation set were extracted from the 80% (60%-20%) of the data and the rest 20% was kept to evaluate the model after its training (*test set*).  

**Create the dataset**: The dataset is created from the raw data in such a format that it can be loader by the *DataLoader* module of torch.utils.data. <br>

In this case, the data was treated as *timeseries* with sequence length equal to 2. In order for that to be able to happen, some data augmentation was performed. Specifically, as noted by the dataset documentation (<a href="https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/RND3SAT/descr.html">Dataset Documentation</a>), for all problem instances there is a clause number-threshold after which the problem is highly unlikely to be satisfiable. So, before this threshold, the problem is most likely to be satisfiable and, for that reason, it is possible to augment the dataset in two ways :
<ol>
    <li>For each instance provide a satisfiable smaller version of it (the instance below the threshold).</li>
    <li>For each unsatisfiable instance provide another unsatisfiable instance (the instance just above the threshold).</li>
</ol>

In [ ]:
pos_weight = dataset_processing()

Start the data processing...



It should be noted the LSTM model needs about 300 times more data that its parameters and in our case we have 970561 parameters and 14092 input data. **Thus, due to the amount of data, even after the augmentation, the model is not expected to reach high values in the evaluation metrics**. 

**Tune parameters**: Tune the parameters of the model (LSTM itself) as well as parameters regarding the training process e.g. *batch-size*, *weight decay* etc. More specidically, the parameters that are tuned are the following.
<ol><li>Model parameters:</li>
    <ul> 
        <li>Number of layers</li>
        <li>Dropout rate</li>
        <li>Hidden Units</li><br>
    </ul>
    <li>Training parameters:</li>
    <ul> 
        <li>Batch size used</li>
        <li>Learning rate</li>
        <li>Weight decay</li>
    </ul>
</ol>

The parameters of the final model are the ones that result in the minimum ***validation error***. <br>
Also, ***early stopping*** is used in order to avoid *overfitting*.

In [ ]:
# Tune parameters
best_parameters = tune_parameters(pos_weight=pos_weight)

# Show best parameters
print(f'Best hyperparameters were: {best_parameters}')
# Store best parameters
with open('./best_parameters_same_sets.txt', 'w') as f:
    f.write(json.dumps(best_parameters))

f.close()

**Train with the best parameters**: The LSTM is trained using the selected parameters.

In [ ]:
# Access the best parameters in order to train final model
with open('best_parameters_same_sets.txt') as f:
    data = f.read()

best_parameters_loaded = json.loads(data)

print('\nNow training with the best parameters\n')
training(params=best_parameters_loaded, make_err_logs=True)

The following Figure shows the ***training and validation set errors***, the epoch where the ***early stopping*** was activated, as well as the epoch of the final ***selected model***.

In [ ]:
display(HTML('<img src="plots/train_valid_error.png?%d" height=400 width=400>' % __counter__))

**Test the model**: Predict the satisfiability of the clauses in the *Test Set* and get the corresponding metrics.

In [ ]:
print('\nResults on the test set:\n')
testing(params=best_parameters_loaded)

The following **Figures** show:
<ol>
<li>The <b>confusion matrix</b> for the test set-prediction</li>
<li>The <b>ROC-AUC curve</b> for the test set-prediction</li>
<li>The <b>precision recall curve</b> for the test set-prediction</li>
</ol>

In [13]:
print("\n1.")
display(HTML('<img src="plots/cm.png?%d" height=500 width=500>' % __counter__))
print("2.")
display(HTML('<img src="plots/roc_auc.png?%d" height=450 width=450>' % __counter__))
print("3.")
display(HTML('<img src="plots/pr.png?%d" height=450 width=450>' % __counter__))


1.


2.


3.


Since, as one can notice, ***the results are not very promising using this model and this architecture***, with the current amount of data, the model is not tested on a Test Set from a different data distribution.